In [ ]:
#@title uploader
file_id = "1rVYGtWzw9s9Bw80-XaIN8gYBsBG79K8B" #@param {type:"string"}
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1gLBqEWEBQDYbKCDigHnUXNTkzl-OslSO

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': file_id }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!unzip {fileId['title']}

newwidsdatathon2025.zip
Archive:  newwidsdatathon2025.zip
  inflating: Data Dictionary.xlsx    
  inflating: SAMPLE_SUBMISSION.xlsx  
  inflating: TEST/TEST_CATEGORICAL.xlsx  
  inflating: TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv  
  inflating: TEST/TEST_QUANTITATIVE_METADATA.xlsx  
  inflating: TRAIN_NEW/TRAINING_SOLUTIONS.xlsx  
  inflating: TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx  
  inflating: TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv  
  inflating: TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx  
  inflating: TRAIN_OLD/TRAINING_SOLUTIONS.xlsx  
  inflating: TRAIN_OLD/TRAIN_CATEGORICAL_METADATA.xlsx  
  inflating: TRAIN_OLD/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv  
  inflating: TRAIN_OLD/TRAIN_QUANTITATIVE_METADATA.xlsx  


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.5 MB/s eta 0:00:00


## Modeling to predict ADHD_Outcome

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
import gc
from scipy.sparse import csr_matrix

# Disable frozen modules warning
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

# ====== CONFIG ======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ====== PREPROCESSING ======
numerical_df = pd.read_excel('TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx').sort_values(by="participant_id").reset_index(drop=True)
categorical_df = pd.read_excel("TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx")
categorical_df.fillna({'PreInt_Demos_Fam_Child_Ethnicity':categorical_df['PreInt_Demos_Fam_Child_Ethnicity'].mean()}, inplace = True)
categorical_df = categorical_df.sort_values(by="participant_id").reset_index(drop=True)
graph_fcm_df = pd.read_csv("TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv").sort_values(by="participant_id").reset_index(drop=True)
train_solutions = pd.read_excel("TRAIN/TRAINING_SOLUTIONS.xlsx").sort_values(by="participant_id").reset_index(drop=True)

labels = train_solutions["ADHD_Outcome"].values
numerical_df = numerical_df.drop(columns=["participant_id"])
categorical_df = categorical_df.drop(columns=["participant_id"])
graph_fcm_df = graph_fcm_df.drop(columns=["participant_id"])

# Normalize numerical features
num_scaler = StandardScaler()
num_features = num_scaler.fit_transform(numerical_df.values)

# One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_features = encoder.fit_transform(categorical_df.values)

# Combine tabular features
tabular_features = np.concatenate([num_features, cat_features], axis=1)
print("Tabular input dim:", tabular_features.shape[1])

# Graph features
connectivity_columns = [col for col in graph_fcm_df.columns if col not in ['participant_id', 'ADHD_Outcome']]
graph_features = graph_fcm_df[connectivity_columns].values

# Create an adjacency matrix (replace with actual adjacency if available)
num_nodes = graph_features.shape[0]  # Ensure correct shape (number of samples, not features)
half_num_nodes = int(num_nodes / 2)  # Half the number of nodes
adjacency_matrix = np.ones((half_num_nodes, half_num_nodes))
np.fill_diagonal(adjacency_matrix, 0)  # No self-loops

edge_index = torch.tensor(np.array(np.where(adjacency_matrix)), dtype=torch.long).to(device)

# Delete unused variables
gc.collect()
# Determine number of nodes (assuming connectome is flattened symmetric matrix)
n_nodes = int(np.sqrt(graph_features.shape[1]))  # e.g., if 1225 -> 35x35

data_list = []
for i in range(len(graph_features)):
    # Reconstruct symmetric connectome matrix
    matrix = graph_features[i].reshape(n_nodes, n_nodes)
    np.fill_diagonal(matrix, 0)  # Optional: remove self-loops

    # Edge list (thresholding or all nonzero entries)
    edge_index_np = np.array(np.nonzero(matrix))
    edge_index = torch.tensor(edge_index_np, dtype=torch.long)

    # Use matrix row values as node features (could also use node degree, PCA, etc.)
    node_features = matrix.copy()
    x = torch.tensor(node_features, dtype=torch.float)  # shape: (n_nodes, n_nodes)

    # Tabular + graph combined
    data = Data(
        x=x,
        edge_index=edge_index,
        y=torch.tensor(labels[i], dtype=torch.long),
        tabular=torch.tensor(tabular_features[i], dtype=torch.float),
    )
    data_list.append(data)


# ====== CLEAN UP ======
del adjacency_matrix
gc.collect()

# ====== DATA SPLIT & LOADERS ======
train_data, test_data = train_test_split(data_list, test_size=0.2, stratify=labels, random_state=42)
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)  # Reduced batch size
test_loader = DataLoader(test_data, batch_size=1)  # Reduced batch size

# ====== MODEL ======
class MultiInputGCN(nn.Module):
    def __init__(self, tabular_input_dim):
        super().__init__()
        self.gcn1 = GCNConv(1, 16)
        self.gcn2 = GCNConv(16, 32)
        self.tabular_net = nn.Sequential(
            nn.Linear(tabular_input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(32 + 32, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))
        graph_embed = global_mean_pool(x, batch)
        tabular_embed = self.tabular_net(data.tabular)
        combined = torch.cat([graph_embed, tabular_embed], dim=1)
        return self.classifier(combined).squeeze(1)

# ====== TRAINING SETUP ======
model = MultiInputGCN(tabular_input_dim=tabular_features.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.BCELoss()
scaler = torch.cuda.amp.GradScaler()  # For mixed precision training

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():  # Mixed precision
            out = model(data)
            loss = criterion(out, data.y.float())
        scaler.scale(loss).backward()  # Scale loss for mixed precision
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        del data, out, loss
        torch.cuda.empty_cache()
        gc.collect()
    print(f"Train Loss: {total_loss / len(train_loader):.4f}")

def evaluate(loader, mode="Test"):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            preds = (out > 0.5).long()
            correct += (preds == data.y).sum().item()
            total += len(data.y)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(data.y.cpu().numpy())
            del data, out, preds
            torch.cuda.empty_cache()
            gc.collect()
    acc = correct / total
    print(f"{mode} Accuracy: {acc:.4f}")
    return all_preds, all_labels

# ====== TRAIN LOOP ======
for epoch in range(1, 21):
    print(f"\nEpoch {epoch}")
    train()
    evaluate(test_loader)

# ====== SAVE MODEL ======
torch.save(model.state_dict(), "multi_input_model_colab.pth")

# ====== INFERENCE FUNCTION ======
def predict_on_new_data(new_tabular, new_graph):
    model.eval()
    with torch.no_grad():
        graph_x = torch.tensor(new_graph.reshape(-1, 1), dtype=torch.float).to(device)
        tabular_x = torch.tensor(new_tabular, dtype=torch.float).to(device)
        data = Data(x=graph_x, edge_index=edge_index.to(device), tabular=tabular_x)
        data.batch = torch.zeros(data.x.shape[0], dtype=torch.long).to(device)
        out = model(data)
        result = (int(out.item() > 0.5), out.item())
        del data, graph_x, tabular_x, out
        torch.cuda.empty_cache()
        gc.collect()
        return result

Using device: cpu
Tabular input dim: 84


ValueError: cannot reshape array of size 19900 into shape (141,141)

## Predicting ADHD Attempt 2 (New Data)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install torch torchvision torchaudio
!pip install torch-geometric torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
!pip install scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def preprocess_data(fcm_path, quant_path, cat_path, solutions_path):
    # Load all datasets
    fcm = pd.read_csv(fcm_path)
    quant = pd.read_excel(quant_path)
    cat = pd.read_excel(cat_path)
    solutions = pd.read_excel(solutions_path)

    # Sort by participant_id to align all datasets
    fcm = fcm.sort_values(by='participant_id').reset_index(drop=True)
    quant = quant.sort_values(by='participant_id').reset_index(drop=True)
    cat = cat.sort_values(by='participant_id').reset_index(drop=True)
    solutions = solutions.sort_values(by='participant_id').reset_index(drop=True)

    # Store participant IDs for tracking (optional)
    participant_ids = fcm['participant_id'].copy()

    # Extract labels and Sex_F for weighting later
    y = solutions['ADHD_Outcome'].values
    sex_f = solutions['Sex_F'].values

    # Drop 'participant_id' and 'Sex_F' from all features
    fcm = fcm.drop(columns=['participant_id'], errors='ignore')
    quant = quant.drop(columns=['participant_id', 'Sex_F'], errors='ignore')
    cat = cat.drop(columns=['participant_id', 'Sex_F'], errors='ignore')

    # Drop rows with missing values in quant & cat
    valid_rows = quant.dropna().index.intersection(cat.dropna().index)

    # Apply valid row filtering to all datasets
    fcm = fcm.loc[valid_rows].reset_index(drop=True)
    quant = quant.loc[valid_rows].reset_index(drop=True)
    cat = cat.loc[valid_rows].reset_index(drop=True)
    y = y[valid_rows]
    sex_f = sex_f[valid_rows]

    # One-hot encode categorical features
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    cat_encoded = encoder.fit_transform(cat)

    # Convert all to numpy
    X_fcm = fcm.values.astype(np.float32)
    X_quant = quant.values.astype(np.float32)
    X_cat = cat_encoded.astype(np.float32)

    return X_quant, X_cat, X_fcm, y, sex_f, participant_ids.loc[valid_rows].tolist()


In [ ]:
fcm, quant, cat, y, sex_f, labels = preprocess_data("TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv",
                                          "TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx",
                                          "TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx",
                                          "TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

In [ ]:
print(fcm.shape)
print(quant.shape)
print(cat.shape)
print(y.shape)
print(sex_f.shape)
print(labels)

(619, 18)
(619, 62)
(619, 19900)
(619,)
(619,)


AttributeError: 'list' object has no attribute 'shape'